# Fashion MNIST Playground

This is me trying to figure out how to get the Fashion MNIST dataset to work so here we go!

### The Real Deal

In [12]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot as plt

# Load the Data, Kronk!
(train_x, train_y), (test_x, test_y) = fashion_mnist.load_data()

# Reshape dataset to have a single color channel. Note "train_x.shape[0]" is 60000.
train_x = train_x.reshape((train_x.shape[0], 28, 28, 1))
test_x = test_x.reshape((test_x.shape[0], 28, 28, 1))

# This is a way to categorize the data - these are the "answers" so to speak, so we want to keep track of them.
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

# Okay... what data value do we try to normalize? We honestly just feed it the "train_x" and "train_x"

# convert from Integers to Floats
train_normalized = train_x.astype('float32')
test_normalized = test_x.astype('float32')

# Normalize to range [0, 1]. I guess this works because it's looking at the entire matrix and then just dividing by 255.0
train_x = train_x.astype('float32') / 255.0
test_x = test_x.astype('float32') / 255.0

# Okay so now that we have all of the data, now we define a model? Is that the next step?
# Well yes kind of, but we also need to set up the K-Folds.
scores = []
histories = []

# Prepare K-Fold Cross Validation
n_folds = 5
kfold = KFold(n_folds, shuffle=True, random_state=1)

# Oh boy... this will be the big one.
for train_x_fold, test_x_fold in kfold.split(train_x):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    # We add the layers here. Need to add the shape first.
    # Also note - play around with this!
    
    # And now we compile the model.
    optimizer = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Select rows for train and test... as I understand this, this is just cycling through the data selecting the test and training data.
    train_X, train_Y, test_X, test_Y = train_x[train_x_fold], train_y[train_x_fold], train_x[test_x_fold], train_y[test_x_fold]
    
    # Fit model... and by fitting the model I think this is just checking to see how accurate it ends up being. Kind of? It's where the model begins to learn.
    history = model.fit(train_X, train_Y, epochs=10, batch_size=32, validation_data=(test_X, test_Y), verbose=0)
    
    # Evaluate model to see how accurate it is.
    _, accuracy = model.evaluate(test_X, test_Y, verbose=0)
    # print('> %.3f' % (accuracy * 100.0))
    
    # Store the scores and accuracy
    scores.append(accuracy)
    histories.append(history)
    
print("Here are the scores.")
print(scores)
print("Here are the histories.")
print(histories)

Here are the scores.
[0.9077500104904175, 0.9102500081062317, 0.9139166474342346, 0.9125000238418579, 0.9104999899864197]
Here are the histories.
[<keras.callbacks.History object at 0x000002BD42B6D940>, <keras.callbacks.History object at 0x000002BD423B25E0>, <keras.callbacks.History object at 0x000002BD263C2730>, <keras.callbacks.History object at 0x000002BD25E02070>, <keras.callbacks.History object at 0x000002BD25EDBD00>]


### Notes

Dataset looks like this:
```
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)
```

So instead of the numbers, it's just articles of clothing. So like... it might just be the same thing?